# 4D emittance measurement data analysis 

This notebook needs to be rewritten. The uncertainty in the measured moments (2%) needs to be shown in the plots.

In [ ]:
import sys
import os
from os.path import join
import copy
import importlib
from tqdm import trange, tqdm

import numpy as np
import pandas as pd
from scipy import optimize as opt
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from matplotlib import animation
import proplot as pplt
import seaborn as sns

from target_image import target_image_analysis

sys.path.append('/Users/46h/Research/')
from scdist.coupling import coupling as BL
from scdist.tools import beam_analysis as ba
from scdist.tools import plotting as myplt
from scdist.tools import utils
from scdist.tools.ap_utils import V_matrix_4x4_uncoupled
from scdist.tools.ap_utils import phase_adv_matrix

from scdist.measurement import analysis
from scdist.measurement import pta
from scdist.measurement.plotting import reconstruction_lines

In [ ]:
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['grid.alpha'] = 0.04
pplt.rc['axes.grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['savefig.transparent'] = True
pplt.rc['savefig.dpi'] = 'figure'
pplt.rc['animation.html'] = 'jshtml'

In [ ]:
eps_labels = [r'$\varepsilon_x$', r'$\varepsilon_y$', 
              r'$\varepsilon_1$', r'$\varepsilon_2$']
format_kws = dict(ygrid=True, xgrid=False)
lineplt_kws = dict(marker='.', alpha=0.8)
savefig_kws = dict(dpi=300)

## Load data 

In [ ]:
# folder = './_saved/2021-09-07/TBT_SCBD_0.5ms/ramp_turns/'
folder = './_saved/2021-09-26/setting1/ramp_turns/'
# folder = './_saved/2021-10-21/setting2/injturns400/ramp_turns/'
# folder = './_saved/2021-08-24/'

# We can restrict the number of measurements used if we want.
max_n_meas = 100 

# We can exclude certain wire-scanners if we want.
exclude = None

In [ ]:
# xlabel = 'Measurement index' # {'Turn number', 'Measurement index'}
xlabel = 'Turn number' # {'Turn number', 'Measurement index'}

if xlabel == 'Turn number':
    xvals = list(range(50, 550, 50))
elif xlabel == 'Measurement index':
    xvals = meas_indices
else:
    raise ValueError('Fix the xlabel.')

In [ ]:
utils.delete_files_not_folders('_output/figures/')

In [ ]:
def load_data_dict(filename, ndarray=True):
    dictionary = dict()
    file = open(filename, 'r')
    for line in file:
        tokens = line.rstrip().split(' ')
        ws_id = tokens[0]
        items = [float(token) for token in tokens[1:]]
        if ws_id not in dictionary:
            dictionary[ws_id] = []
        dictionary[ws_id].append(items)
    file.close()
    dictionary = utils.blacklist(dictionary, exclude)
    for ws_id in dictionary:
        if max_n_meas:
            dictionary[ws_id] = dictionary[ws_id][:max_n_meas]
        if ndarray:
            dictionary[ws_id] = np.array(dictionary[ws_id])
    return dictionary

In [ ]:
filenames = utils.list_files(os.path.join(folder, 'profiles/'))
measurements = pta.read_files(filenames)
moments_dict = pta.get_moments_dict(measurements)

In [ ]:
tmats_dict = load_data_dict(os.path.join(folder, 'info/transfer_mats.dat'), ndarray=False)
# moments_dict = load_data_dict(os.path.join(folder, 'info/moments.dat'))
phases_dict = load_data_dict(os.path.join(folder, 'info/phase_adv.dat'))
pos_x_dict = load_data_dict(os.path.join(folder, 'info/pos_x.dat'))
pos_y_dict = load_data_dict(os.path.join(folder, 'info/pos_y.dat'))
pos_u_dict = load_data_dict(os.path.join(folder, 'info/pos_u.dat'))
raw_x_dict = load_data_dict(os.path.join(folder, 'info/raw_x.dat'))
raw_y_dict = load_data_dict(os.path.join(folder, 'info/raw_y.dat'))
raw_u_dict = load_data_dict(os.path.join(folder, 'info/raw_u.dat'))

ws_ids = sorted(list(tmats_dict))
n_meas = len(tmats_dict[ws_ids[0]])
meas_indices = list(range(n_meas))

# Convert from 16 element lists to 4x4 arrays for transfer matrices.
for ws_id in ws_ids:
    for i in range(n_meas):
        tmats_dict[ws_id][i] = np.reshape(tmats_dict[ws_id][i], (4, 4))

In [ ]:
# r = 0.01
# dphi = np.radians(0.5)

# sig_xx, sig_yy, sig_xy = moments.T
# phi = np.radians(45.0)
# sig_uu = 0.5 * (sig_xx + sig_yy) + sig_xy

# sig_xx_err = r * moments[:, 0]
# sig_yy_err = r * moments[:, 1]
# sig_xy_err = np.sqrt(r**2 * (sig_uu**2 + 0.25 * (sig_xx**2 + sig_yy**2)) + ((sig_xx - sig_yy) * dphi)**2)

In [ ]:
fig, axes = pplt.subplots(nrows=3, figsize=(4.25, 4.5), spany=False, wspace=1, aligny=True)
axes.format(cycle='538')
# plot_kws = dict(marker='.')
plot_kws = dict(marker='.', capsize=0)
handles = []
for ws_id in ws_ids:    
    sig_xx_list = np.array([measurement[ws_id].hor.stats['Sigma'].rms**2 for measurement in measurements])
    sig_yy_list = np.array([measurement[ws_id].ver.stats['Sigma'].rms**2 for measurement in measurements])
    sig_uu_list = np.array([measurement[ws_id].dia.stats['Sigma'].rms**2 for measurement in measurements])
    sig_xy_list = sig_uu_list - 0.5 * (sig_xx_list + sig_yy_list)
    f = 0.02
    dsig_xx_list = f * sig_xx_list
    dsig_yy_list = f * sig_yy_list
    dsig_uu_list = f * sig_uu_list
    dsig_xy_list = np.sqrt(dsig_uu_list + 0.25 * (dsig_xx_list**2 + dsig_yy_list**2))
    axes[0].errorbar(meas_indices, sig_xx_list, yerr=dsig_xx_list, **lineplt_kws)
    axes[1].errorbar(meas_indices, sig_yy_list, yerr=dsig_yy_list, **lineplt_kws)
    h = axes[2].errorbar(meas_indices, sig_xy_list, yerr=dsig_xy_list, **lineplt_kws)
    
    handles.append(h)
# handles = [Line2D([0], [0], color=c, **lineplt_kws) for c in ]
axes[0].legend(handles, labels=ws_ids, ncols=1, loc='r', fontsize='small')


    
#     axes[0].plot(meas_indices, moments[:, 0], **plot_kws)    
#     axes[1].plot(meas_indices, moments[:, 1], **plot_kws)
#     h = axes[2].plot(meas_indices, moments[:, 2], **plot_kws)
axes[0].format(ylabel=r'$\langle{x^2}\rangle$ [mm$^2$]')
axes[1].format(ylabel=r'$\langle{y^2}\rangle$ [mm$^2$]')
axes[2].format(ylabel=r'$\langle{xy}\rangle$ [mm$^2$]')
axes.format(xlabel='Measurement index', xtickminor=False, xticks=meas_indices)
axes[0].set_title('Measured moments', fontsize='medium')
plt.savefig('_output/figures/moments.png', **savefig_kws)

In [ ]:
for ws_id in ws_ids:
    moments = []
    for measurement in measurements:
        sig_xx = measurement[ws_id].hor.stats['Sigma'].rms**2
        sig_yy = measurement[ws_id].ver.stats['Sigma'].rms**2
        sig_uu = measurement[ws_id].dia.stats['Sigma'].rms**2
        moments.append([sig_xx, sig_yy, sig_uu])
    moments = np.array(moments)
    std = np.std(moments, axis=0)
    print('Standard deviations [mm^2 mrad^2]:', std)
    f = np.abs(np.max(moments, axis=0) - np.min(moments, axis=0)) / np.min(moments, axis=0)
    print('Absolute fractional changes:', f)
    print('Average absolute fractional change:', np.mean(f))
    print()

In [ ]:
fig, axes = pplt.subplots(ncols=3, figsize=(7.5, 2.1), spanx=False, sharey=False, alignx=True)
plot_kws = dict(marker='.')
axes.format(cycle='538')
for ws_id in ws_ids:
    moments = moments_dict[ws_id]
    axes[0].plot(xvals, moments[:, 0], **plot_kws)    
    axes[1].plot(xvals, moments[:, 1], **plot_kws)
    axes[2].plot(xvals, moments[:, 2], **plot_kws)
axes[0].format(ylabel=r'$\langle{x^2}\rangle$ [mm$^2$]')
axes[1].format(ylabel=r'$\langle{y^2}\rangle$ [mm$^2$]')
axes[2].format(ylabel=r'$\langle{xy}\rangle$ [mm$^2$]')
axes[0].set_ylim(0., max(axes[0].get_ylim()[1], axes[1].get_ylim()[1]))
axes[1].set_ylim(0., max(axes[0].get_ylim()[1], axes[1].get_ylim()[1]))
ymax = max(np.abs(axes[2].get_ylim()))
axes[2].set_ylim(-ymax, ymax)
axes.format(xlabel=xlabel, **format_kws)
plt.savefig('_output/figures/moments_hor.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(ncols=3, figsize=(7.5, 2.1), spanx=False, sharey=False, alignx=True)
plot_kws = dict(marker='.')
axes.format(cycle='538')
for ws_id in ws_ids:
    moments = moments_dict[ws_id]
    axes[0].plot(xvals, np.sqrt(moments[:, 0]), **plot_kws)    
    axes[1].plot(xvals, np.sqrt(moments[:, 1]), **plot_kws)
    axes[2].plot(xvals, moments[:, 2] / np.sqrt(moments[:, 0] * moments[:, 1]), **plot_kws)
axes[0].format(ylabel=r'$\sqrt{\langle{x^2}\rangle}$ [mm]')
axes[1].format(ylabel=r'$\sqrt{\langle{y^2}\rangle}$ [mm]', )
axes[2].format(ylabel=r'$x$-$y$ cor. coef.')
axes[0].set_ylim(0., max(axes[0].get_ylim()[1], axes[1].get_ylim()[1]))
axes[1].set_ylim(0., max(axes[0].get_ylim()[1], axes[1].get_ylim()[1]))
ymax = max(np.abs(axes[2].get_ylim()))
axes[2].set_ylim(-ymax, ymax)
axes.format(xlabel=xlabel, **format_kws)
plt.savefig('_output/figures/moments_corr_hor.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(ncols=2, figsize=(6, 2.5))
axes.format(cycle='538')
handles = []
for ws_id in ws_ids:
    mux, muy = phases_dict[ws_id].T
    axes[0].plot(xvals, mux, **plot_kws)
    h = axes[1].plot(xvals, muy, **plot_kws)
    handles.append(h)
axes.format(xlabel=xlabel, ylabel='[rad]', ylim=(0, 2 * np.pi),
            xtickminor=False, xticks=range(n_meas))
axes[0].set_title('Horizontal phase adv.')
axes[1].set_title('Vertical phase adv.')
axes[1].legend(handles, ws_ids, ncols=1, loc='r', fontsize='small')
axes.format(**format_kws)
plt.savefig('_output/figures/phases.png', **savefig_kws)

In [ ]:
# fig, axes = pplt.subplots(ncols=3)
# colors = ['red8', 'blue8', 'green8']
# for i in range(len(ws_ids) - 1):
#     diffs = phases_dict[ws_ids[i + 1]] - phases_dict[ws_ids[i]]
#     for j in range(len(diffs)):
#         for k in range(2):
#             if diffs[j, k] < 0:
#                 diffs[j, k] += 2 * np.pi
#     axes[0].plot(np.degrees(diffs[:, 0]), color=colors[i], marker='.')
#     axes[1].plot(np.degrees(diffs[:, 1]), color=colors[i], marker='.')
#     axes[2].plot(np.degrees(np.abs(diffs[:, 0] - diffs[:, 1])), color=colors[i], marker='.')
# for ax in axes:
#     ax.grid(axis='y')
# labels = [r'{} $\rightarrow$ {}'.format(ws_ids[i], ws_ids[i + 1]) for i in range(len(ws_ids) - 1)]
# axes[2].legend(labels=labels, ncol=1, loc=(1.02, 0))
# axes[0].set_title(r'Horizontal: $\Delta\mu_x$')
# axes[1].set_title(r'Vertical: $\Delta\mu_y$')
# axes[2].set_title(r'Difference: $\left| \Delta\mu_x - \Delta\mu_y \right|$')
# axes.format(ylabel='Gap', yformatter='deg', suptitle='Wire-scanner phase spacing',
#             xlabel='Measurement index', xtickminor=False, xticks=range(n_meas))
# plt.savefig('_output/figures/phase_gaps.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(ncols=4, figsize=(8, 2))
xmax = 0.
for ax, ws_id in zip(axes, ws_ids):
    ax.set_title(ws_id, fontsize='small')
    for sig_xx, sig_yy, sig_xy in moments_dict[ws_id]:
        angle, c1, c2 = ba._rms_ellipse_dims(sig_xx, sig_yy, sig_xy)
        xmax = max(xmax, c1, c2)
        myplt.ellipse(ax, c1, c2, angle)
xmax *= 1.5
axes.format(xlim=(-xmax, xmax), ylim=(-xmax, xmax), 
            xlabel='x [mm]', ylabel='y [mm]',
            suptitle='Measured rms ellipses in x-y plane')
plt.savefig('_output/figures/corr.png', **savefig_kws)

In [ ]:
# fig, axes = pplt.subplots(ncols=3, figsize=(8, 2.0), sharex=False)
# handles = []
# ws_id = 'RTBT_Diag:WS24'

# meas_index = 0
# plot_kws = dict(marker='.', ms=0, color='black')
# axes[0].plot(pos_x_dict[ws_id][meas_index], raw_x_dict[ws_id][meas_index], **plot_kws)
# axes[1].plot(pos_y_dict[ws_id][meas_index], raw_y_dict[ws_id][meas_index], **plot_kws)
# g1 = axes[2].plot(pos_u_dict[ws_id][meas_index], raw_u_dict[ws_id][meas_index], **plot_kws)

# meas_index = 4
# plot_kws = dict(marker='.', lw=0, color='red', ms=1.75)
# axes[0].plot(pos_x_dict[ws_id][meas_index], raw_x_dict[ws_id][meas_index], **plot_kws)
# axes[1].plot(pos_y_dict[ws_id][meas_index], raw_y_dict[ws_id][meas_index], **plot_kws)
# g2 = axes[2].plot(pos_u_dict[ws_id][meas_index], raw_u_dict[ws_id][meas_index], **plot_kws)

# axes[0].format(xlabel='x [mm]')
# axes[1].format(xlabel='y [mm]')
# axes[2].format(xlabel='u [mm]')
# for ax in axes:
#     ax.grid(axis='y')

# axes[2].legend([g1, g2], ['Measurment 1', 'Measurement 2'], loc='top', fontsize='medium', framealpha=0)
# plt.savefig('_output/figures/WS comparison.png', **savefig_kws)

In [ ]:
ws_plot_kws = dict(marker='.', ms=3, alpha=0.7)
width = 120.
xcenter = 120.
ycenter = 95.
ucenter = 100.
xlim_x = (xcenter - 0.5 * width, xcenter + 0.5 * width)
xlim_y = (ycenter - 0.5 * width, ycenter + 0.5 * width)
xlim_u = (ucenter - 0.5 * width, ucenter + 0.5 * width)

# for same_ylim in [False, True]:

#     # Get global maximum signal height.
#     ymax = 0.
#     for meas_index in range(n_meas):
#         measurement = measurements[meas_index]
#         for ws_id in ws_ids:
#             fx = measurement[ws_id].hor.raw
#             fy = measurement[ws_id].ver.raw
#             fu = measurement[ws_id].dia.raw            
#             ymax = max(ymax, max(np.max(fx), np.max(fy), np.max(fu)))

#     for meas_index in range(n_meas):
#         measurement = measurements[meas_index]
#         fig, axes = pplt.subplots(ncols=3, figsize=(8, 2.0), sharex=False)
#         axes.format(cycle='538', rc_kw={'legend.fontsize': 5}, **format_kws)
#         handles = []
#         for ws_id in ws_ids:
#             axes[0].plot(measurement[ws_id].hor.pos, measurement[ws_id].hor.raw, **ws_plot_kws)
#             axes[1].plot(measurement[ws_id].ver.pos, measurement[ws_id].ver.raw, **ws_plot_kws)
#             g = axes[2].plot(measurement[ws_id].dia.pos, measurement[ws_id].dia.raw, **ws_plot_kws)            
#             handles.append(g)
#         axes[0].format(xlabel='x [mm]')
#         axes[1].format(xlabel='y [mm]')
#         axes[2].format(xlabel='u [mm]')
#         if same_ylim:
#             ymin = axes[0].get_ylim()[0]
#             axes.format(ylim=(ymin, ymax))
#         axes[0].set_xlim(xlim_x)
#         axes[1].set_xlim(xlim_y)
#         axes[2].set_xlim(xlim_u)
#         axes[2].legend(handles, labels=ws_ids, ncols=1, loc='r')
        
#         if xlabel == 'Turn number':
#             plt.suptitle('Turn = {}'.format(xvals[meas_index]))
#         else:
#             plt.suptitle('Measurement index = {}'.format(meas_index))
        
#         if same_ylim:
#             figname = '_output/figures/profiles_sameylim_{}.png'.format(meas_index)
#         else:
#             figname = '_output/figures/profiles_{}.png'.format(meas_index)
#         plt.savefig(figname, **savefig_kws)
#         plt.show()
        
#     print()
#     print()

In [ ]:
meas_index = -1
measurement = measurements[-1]
colors = pplt.Cycle('538').by_key()['color']

fig, axes = pplt.subplots(nrows=2, ncols=3, figsize=(8.0, 4.25), sharex=False, sharey=False, height_ratios=[1.0, 1.25], hspace=5.0)
axes.format(cycle='538', rc_kw={'legend.fontsize': 'small'}, **format_kws)
handles = []
for ws_id in ws_ids:
    axes[0, 0].plot(measurement[ws_id].hor.pos, measurement[ws_id].hor.raw, **ws_plot_kws)
    axes[0, 1].plot(measurement[ws_id].ver.pos, measurement[ws_id].ver.raw, **ws_plot_kws)
    g = axes[0, 2].plot(measurement[ws_id].dia.pos, measurement[ws_id].dia.raw, **ws_plot_kws)
    handles.append(g)
    axes[0, 0].set_xlabel('x [mm]', fontsize='large')
    axes[0, 1].set_xlabel('y [mm]', fontsize='large')
    axes[0, 2].set_xlabel('u [mm]', fontsize='large')
axes[0, 0].set_xlim(xlim_x)
axes[0, 1].set_xlim(xlim_y)
axes[0, 2].set_xlim(xlim_u)
    
# pplt.rc['legend.fontsize'] = 8 # 'fontsize' doesn't work in the plot call!
axes[0, 2].legend(handles, 
                  labels=[w.split(':')[-1] for w in ws_ids], 
                  ncols=2, loc='top', handlelength=1.5)

kws = dict(marker='.')
for ws_id, color in zip(ws_ids, colors):
    axes[1, 0].plot(xvals, np.sqrt(moments_dict[ws_id][:, 0]), **kws)
    axes[1, 1].plot(xvals, np.sqrt(moments_dict[ws_id][:, 1]), **kws)
    axes[1, 2].plot(xvals, moments_dict[ws_id][:, 2] / np.sqrt(moments_dict[ws_id][:, 0] * moments_dict[ws_id][:, 1]), **kws)
axes[0, 0].set_ylabel('Signal', fontsize='large')
axes[0, 1:].format(yticklabels=[])
axes[1, 2].format(ylim=(-0.5, 0.5))


axes[1, :2].format(ylim=(0., max(ax.get_ylim()[1] for ax in axes[1, :2])))
axes[1, 0].set_ylabel('[mm]')

axes[1, 0].set_ylabel(r'$\sqrt{\langle{xx}\rangle}$ [mm]', fontsize='large')
axes[1, 1].set_ylabel(r'$\sqrt{\langle{yy}\rangle}$ [mm]', fontsize='large')
axes[1, 2].set_ylabel('x-y corr. coef.', fontsize='large')
for ax in axes[1, :]:
    ax.set_xlabel('Turn number', fontsize='large')
    ax.set_xlim(0, ax.get_xlim()[1])
    
plt.savefig('_output/figures/wirescanner_measurements.png', **savefig_kws)

In [ ]:
# emittances = []
# nkeep = list(range(1, n_meas))
# for j in nkeep:
#     tmats_list, moments_list = [], []
#     for ws_id in ws_ids:
#         tmats_list.extend(tmats_dict[ws_id][:j])
#         moments_list.extend(moments_dict[ws_id][:j])
#     Sigma = reconstruct(tmats_list, moments_list)
#     emittances.append(ba.emittances(Sigma))
# emittances = np.array(emittances)
    
# fig, ax = pplt.subplots()
# ax.plot(nkeep, np.sqrt(emittances[:, 0] * emittances[:, 1]) - np.sqrt(eps_x * eps_y), 
#         marker='.', color='red8')
# ax.plot(nkeep, np.sqrt(emittances[:, 2] * emittances[:, 3]) - np.sqrt(eps_1 * eps_2),
#         marker='.', color='blue8')
# ax.legend(labels=[r'$\sqrt{\varepsilon_x\varepsilon_y}$', r'$\sqrt{\varepsilon_1\varepsilon_2}$'], ncols=2)
# ax.format(xlabel='Number of measurements used', ylabel='Difference from full fit [mm mrad]')
# plt.savefig('_output/figures/nmeas.png', facecolor='white', dpi=300)

## Reconstruction from all measurements

This section assumes that all measurements correspond to the same beam. Ignore it if that is not true.

In [ ]:
tmats_list, moments_list = [], []
for ws_id in ws_ids:
    tmats_list.extend(tmats_dict[ws_id])
    moments_list.extend(moments_dict[ws_id])

In [ ]:
Sigma, C = analysis.reconstruct(tmats_list, moments_list)

Corr = utils.cov2corr(Sigma)
alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
eps_x, eps_y, eps_1, eps_2 = ba.emittances(Sigma)
eps_4D = eps_1 * eps_2
coupling_coeff = 1.0 - np.sqrt((eps_1 * eps_2) / (eps_x * eps_y))

In [ ]:
Sigma_std = analysis.to_mat(np.sqrt(C.diagonal()))
eps_x_std, eps_y_std, eps_1_std, eps_2_std = analysis.propagate_emittance_errors(Sigma, C)
alpha_x_std, alpha_y_std, beta_x_std, beta_y_std = analysis.propagate_twiss_errors(Sigma, C)
eps_4D_std = analysis.get_eps4D_std(eps_1, eps_2, eps_1_std, eps_2_std)

In [ ]:
print('Sigma =')
print(Sigma)
print('Standard deviation in Sigma from LLSQ =')
print(Sigma_std)
print('Corr =')
print(Corr)
print('eps_4D = {:.3f} +- {:.3f} [mm^2 mrad^2]'.format(eps_4D, eps_4D_std))
print('eps_1 = {:.3f} +- {:.3f} [mm mrad]'.format(eps_1, eps_1_std))
print('eps_2 = {:.3f} +- {:.3f} [mm mrad]'.format(eps_2, eps_2_std))
print('eps_x = {:.3f} +- {:.3f} [mm mrad]'.format(eps_x, eps_x_std))
print('eps_y = {:.3f} +- {:.3f} [mm mrad]'.format(eps_y, eps_y_std))
print('beta_x = {:.3f} +- {:.3f} [m/rad]'.format(beta_x, beta_x_std))
print('beta_y = {:.3f} +- {:.3f} [m/rad]'.format(beta_y, beta_y_std))
print('alpha_x = {:.3f} +- {:.3f} [m/rad]'.format(alpha_x, alpha_x_std))
print('alpha_y = {:.3f} +- {:.3f} [m/rad]'.format(alpha_y, alpha_y_std))
print('Coupling coefficient = {}'.format(coupling_coeff))

In [ ]:
norm = '2D'
V = np.identity(4)
if norm == '2D':
    alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
    V = V_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
elif norm == '4D':
    U = np.array([[0, 1, 0, 0], [-1, 0, 0, 0], [0, 0, 0, 1], [0, 0, -1, 0]])
    eigvals, eigvecs = np.linalg.eig(np.matmul(Sigma, U))
    V = BL.construct_V(eigvecs)
Vinv = np.linalg.inv(V)
Sigma_n = np.linalg.multi_dot([Vinv, Sigma, Vinv.T])

In [ ]:
axes = myplt.rms_ellipses(Sigma, color='black', pad=0.75, alpha=0.15, fill=True, lw=0, zorder=0)
plt.suptitle('Reconstructed phase space')
reconstruction_lines(axes[2, 2], tmats_dict, moments_dict, plane='y-yp')
reconstruction_lines(axes[0, 0], tmats_dict, moments_dict, plane='x-xp',
                     legend=True, legend_kws=dict(loc=(1.15, 0)))
plt.savefig('_output/figures/corner.png', **savefig_kws)

In [ ]:
axes = myplt.rms_ellipses(Sigma_n, color='black', pad=0.75, alpha=0.15, fill=True, lw=0, zorder=0)
plt.suptitle('Reconstructed phase space (normalized {})'.format(norm))
reconstruction_lines(axes[2, 2], tmats_dict, moments_dict, plane='y-yp', norm_mat=V)
reconstruction_lines(axes[0, 0], tmats_dict, moments_dict, plane='x-xp', norm_mat=V,
                     legend=True, legend_kws=dict(loc=(1.15, 0)))
labels = ["x [mm]", "x' [mrad]", "y [mm]", "y' [mrad]"]
for i in range(3):
    axes[i, 0].set_ylabel(labels[i + 1])
    axes[-1, i].set_xlabel(labels[i])
plt.savefig('_output/figures/corner_norm.png', **savefig_kws)

In [ ]:
# ellipse_kws = dict(color="black", alpha=0.15, fill=True, lw=0, zorder=0)
# line_kws = dict(lw=1.1)

# fig, axes = pplt.subplots(ncols=2, figsize=None, spanx=False, spany=False, sharex=False, sharey=False)
# reconstruction_lines(axes[0], tmats_dict, moments_dict, plane="x-xp", norm_mat=V, 
#                      legend=True, legend_kws=dict(loc='top', framealpha=0., ncol=2), **line_kws
#                     )
# reconstruction_lines(axes[1], tmats_dict, moments_dict, plane="y-yp", norm_mat=V, **line_kws)
# angle, c1, c2 = ba.rms_ellipse_dims(Sigma_n, "x", "xp")
# myplt.ellipse(axes[0], c1, c2, angle, **ellipse_kws)
# angle, c1, c2 = ba.rms_ellipse_dims(Sigma_n, "y", "yp")
# myplt.ellipse(axes[1], c1, c2, angle, **ellipse_kws)
# axes.format(xlim=(-10, 10), ylim=(-10, 10))
# label_kws = dict(fontsize=12)
# axes[0].set_xlabel(r"$x_n$", **label_kws)
# axes[0].set_ylabel(r"$x'_n$", **label_kws)
# axes[1].set_xlabel(r"$y_n$", **label_kws)
# axes[1].set_ylabel(r"$y'_n$", **label_kws)
# axes.format(aspect=1)
# plt.savefig('_output/figures/rec_lines.png', **savefig_kws)

## Reconstruction at each measurement

In [ ]:
Sigmas = []
Cmats = []
for meas_index in range(n_meas):
    transfer_matrices, moments = [], []
    for ws_id in ws_ids:
        transfer_matrices.append(tmats_dict[ws_id][meas_index])
        moments.append(moments_dict[ws_id][meas_index])
    Sigma, C = analysis.reconstruct(transfer_matrices, moments)
    Sigmas.append(Sigma)
    Cmats.append(C)

In [ ]:
stats = ba.BeamStats()
stats.read_moments(np.array([Sigma[np.triu_indices(4)] for Sigma in Sigmas]))

In [ ]:
Sigma_stds = np.array([analysis.to_mat(np.sqrt(C.diagonal())) for C in Cmats])
emittance_stds = np.array([analysis.propagate_emittance_errors(Sigma, C) 
                           for Sigma, C in zip(Sigmas, Cmats)])
twiss_stds = np.array([analysis.propagate_twiss_errors(Sigma, C) 
                       for Sigma, C in zip(Sigmas, Cmats)])
eps_4D_stds = np.sqrt((emittance_stds[:, 2] * stats.twiss4D['eps_2'])**2 + (emittance_stds[:, 3] * stats.twiss4D['eps_1'])**2)
eps_4D_apparent_stds = np.sqrt((emittance_stds[:, 0] * stats.twiss2D['eps_y'])**2 + (emittance_stds[:, 1] * stats.twiss2D['eps_x'])**2)

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
plt_kws = dict(marker='.', capsize=1.5)
g1 = ax.errorbar(xvals, stats.twiss2D['eps_x'], yerr=emittance_stds[:, 0], **plt_kws)
g2 = ax.errorbar(xvals, stats.twiss2D['eps_y'], yerr=emittance_stds[:, 1], **plt_kws)

# Hard-code the fractional intrinsic emittance error for now. 
frac_err = 0.04
g3 = ax.errorbar(xvals, stats.twiss4D['eps_1'], yerr=frac_err * stats.twiss4D['eps_1'], **plt_kws)
g4 = ax.errorbar(xvals, stats.twiss4D['eps_2'], yerr=frac_err * stats.twiss4D['eps_2'], **plt_kws)

lkws = dict(marker='.')
lines = [
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[0], **lkws),
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[1], **lkws),
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[2], **lkws),
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[3], **lkws),
]
ax.legend(lines, labels=eps_labels, loc='upper left', ncols=1, handlelength=1.5)
ax.format(xlabel=xlabel, ylabel='[mm mrad]', 
          ylim=(0., ax.get_ylim()[1]), 
          xlim=(0, ax.get_xlim()[1]),
          **format_kws
         )
# ax.format(ylim=(0., 12.2))
plt.savefig('_output/figures/emittances.png', **savefig_kws)

### Adding errors

These are the absolute differences between the xx, yy, and xy moments for the two measurements we took at 400 turns at each wire-scanner.

In [ ]:
meas1 = pta.Measurement('_saved/2021-10-21/setting2/injturns400/turn400/WireAnalysisFmt-2021.10.21_19.22.35.pta.txt')
meas2 = pta.Measurement('_saved/2021-10-21/setting2/injturns400/ramp_turns/profiles/WireAnalysisFmt-2021.10.21_20.32.41.pta.txt')
moments_dict1 = meas1.get_moments()
moments_dict2 = meas2.get_moments()
deltas = dict()
for ws_id in ws_ids:    
    sig_xx1, sig_yy1, sig_xy1 = moments_dict1[ws_id]
    sig_xx2, sig_yy2, sig_xy2 = moments_dict2[ws_id]
    delta_sig_xx = abs(sig_xx1 - sig_xx2)
    delta_sig_yy = abs(sig_yy1 - sig_yy2)
    delta_sig_xy = abs(sig_xy1 - sig_xy2)
    deltas[ws_id] = [delta_sig_xx, delta_sig_yy, delta_sig_xy]

In [ ]:
for ws_id in ws_ids:
    print(ws_id, deltas[ws_id])

Plot the profiles.

In [ ]:
for ws_id in ws_ids:
    profile1 = meas1[ws_id]
    profile2 = meas2[ws_id]
    fig, axes = pplt.subplots(ncols=3, figsize=(8, 2))
    plot_kws1 = dict(lw=1.0)
    plot_kws2 = dict(lw=0.0, marker='.', alpha=0.7, ms=3)
    axes.format(suptitle=ws_id)
    axes[0].plot(profile1.hor.pos, profile1.hor.raw, color='black', **plot_kws1)
    axes[0].plot(profile2.hor.pos, profile2.hor.raw, color='red', **plot_kws2)

    axes[1].plot(profile1.ver.pos, profile1.ver.raw, color='black', **plot_kws1)
    axes[1].plot(profile2.ver.pos, profile2.ver.raw, color='red', **plot_kws2)

    axes[2].plot(profile1.dia.pos, profile1.dia.raw, color='black', **plot_kws1)
    axes[2].plot(profile2.dia.pos, profile2.dia.raw, color='red', **plot_kws2)

    axes[0].set_title('Horizontal')
    axes[1].set_title('Vertical')
    axes[2].set_title('Diagonal')
    
    annotate_kws = dict(xycoords='axes fraction', fontsize='small')
    xy1 = (0.01, 0.92)
    xy2 = (0.01, 0.82)
    xy3 = (0.01, 0.75)
    sig1 = meas1[ws_id].hor.stats['Sigma'].rms
    sig2 = meas2[ws_id].hor.stats['Sigma'].rms
    abs_err = abs((sig1**2 - sig2**2))
    abs_frac_err = abs_err / (min(sig1, sig2)**2)
    axes[0].annotate(r'$\sigma_1^2$ = {:.3f}'.format(sig1**2), xy=xy1, **annotate_kws)
    axes[0].annotate(r'$\sigma_2^2$ = {:.3f}'.format(sig2**2), xy=xy2, **annotate_kws)
    axes[0].annotate('abs. frac. err. = {:.3f}'.format(abs_frac_err), xy=xy3, **annotate_kws)
    sig1 = meas1[ws_id].ver.stats['Sigma'].rms
    sig2 = meas2[ws_id].ver.stats['Sigma'].rms
    abs_err = abs((sig1**2 - sig2**2))
    abs_frac_err = abs_err / (min(sig1, sig2)**2)
    axes[1].annotate(r'$\sigma_1^2$ = {:.3f}'.format(sig1**2), xy=xy1, **annotate_kws)
    axes[1].annotate(r'$\sigma_2^2$ = {:.3f}'.format(sig2**2), xy=xy2, **annotate_kws)
    axes[1].annotate('abs. frac. err. = {:.3f}'.format(abs_frac_err), xy=xy3, **annotate_kws)
    sig1 = meas1[ws_id].dia.stats['Sigma'].rms
    sig2 = meas2[ws_id].dia.stats['Sigma'].rms
    abs_err = abs((sig1**2 - sig2**2))
    abs_frac_err = abs_err / (min(sig1, sig2)**2)
    axes[2].annotate(r'$\sigma_1^2$ = {:.3f}'.format(sig1**2), xy=xy1, **annotate_kws)
    axes[2].annotate(r'$\sigma_2^2$ = {:.3f}'.format(sig2**2), xy=xy2, **annotate_kws)
    axes[2].annotate('abs. frac. err. = {:.3f}'.format(abs_frac_err), xy=xy3, **annotate_kws)
    
    plt.savefig('_output/figures/profiles_{}.png'.format(ws_id), facecolor='white', **savefig_kws)

#     dx = 60
#     mean = profile1.hor.stats['Mean'].rms
#     axes[0].set_xlim(mean - dx, mean + dx)

#     mean = profile1.ver.stats['Mean'].rms
#     axes[1].set_xlim(mean - dx, mean + dx)

#     mean = profile1.dia.stats['Mean'].rms
#     axes[2].set_xlim(mean - dx, mean + dx)

I think the xy calculation is pretty sensitive. The biggest error there is at WS21, when the difference in xx moments is largest. We will now add random errors to the measurements.

In [ ]:
n_trials = 5000
emittances_list = []
emittances_mean_list = []
emittances_std_list = []
Sigmas_mean_list = []
Sigmas_std_list = []
turns = xvals

# Set minimum and maximum errors on measured moments.
err_sig_xx_max = 5.0 # [mm mrad]
err_sig_yy_max = 5.0 # [mm mrad]
err_sig_uu_max = 5.0 # [mm mrad]
err_sig_xx_min = 2.0 # [mm mrad]
err_sig_yy_min = 2.0 # [mm mrad]
err_sig_uu_min = 2.0 # [mm mrad]
    
# Collect transfer matrices and measured moments.
for meas_index in meas_indices:
    measurement = measurements[meas_index]
    transfer_matrices, moments = [], []
    for ws_id in ws_ids:
        transfer_matrices.append(tmats_dict[ws_id][meas_index])
        sig_xx = (measurement[ws_id].hor.stats['Sigma'].rms)**2
        sig_yy = (measurement[ws_id].ver.stats['Sigma'].rms)**2
        sig_uu = (measurement[ws_id].dia.stats['Sigma'].rms)**2
        moments.append([sig_xx, sig_yy, sig_uu])
    moments = np.array(moments)

    # Reconstruct with noise.  
#     ws_noise_index = 'all'
#     reduce = float(turns[meas_index]) / turns[-1]
#     err_sig_xx = max(err_sig_xx_min, err_sig_xx_max * reduce)
#     err_sig_yy = max(err_sig_yy_min, err_sig_yy_max * reduce)
#     err_sig_uu = max(err_sig_uu_min, err_sig_uu_max * reduce)
#     errs = np.array([err_sig_xx, err_sig_yy, err_sig_uu])
#     print('max random errors:', errs)

    f = 0.02
    lo = 1.0 - f
    hi = 1.0 + f
    
    # Monte Carlo
    def run_trial():
        _moments = np.random.uniform(lo * moments, hi * moments, size=moments.shape)
#         _moments = moments * (1.0 + np.random.normal(scale=f, size=moments.shape))
        _moments[:, 2] = pta.get_sig_xy(_moments[:, 0], _moments[:, 1], _moments[:, 2], pta.DIAG_WIRE_ANGLE)
        Sigma, C = analysis.reconstruct(transfer_matrices, _moments)
        return Sigma
    
    emittances = []
    Sigmas = []
    fails = 0
    for _ in trange(n_trials):
        failed = True
        while failed:
            Sigma = run_trial()
            failed = not analysis.is_physical_cov(Sigma)
            if failed:
                fails += 1
        emittances.append(ba.emittances(Sigma))
        Sigmas.append(Sigma)
        
    emittances = np.array(emittances)
    emittances_mean = np.mean(emittances, axis=0)
    emittances_std = np.std(emittances, axis=0)
    Sigmas = np.array(Sigmas)
    Sigmas_mean = np.mean(Sigmas, axis=0)
    Sigmas_std = np.std(Sigmas, axis=0)
    print('means:', emittances_mean)
    print('stds:', emittances_std)
    print('fails = {}'.format(fails))
    
    emittances_list.append(emittances)
    emittances_mean_list.append(emittances_mean)
    emittances_std_list.append(emittances_std)
    Sigmas_mean_list.append(Sigmas_mean)
    Sigmas_std_list.append(Sigmas_std)
    
emittances_list = np.array(emittances_list)
emittances_mean_list = np.array(emittances_mean_list)
emittances_std_list = np.array(emittances_std_list)
Sigmas_mean_list = np.array(Sigmas_mean_list)
Sigmas_std_list = np.array(Sigmas_std_list)

In [ ]:
for meas_index in meas_indices:
    emittances = emittances_list[meas_index]
    g = sns.pairplot(pd.DataFrame(emittances, columns=[r'$\varepsilon_x$', r'$\varepsilon_y$', 
                                                       r'$\varepsilon_1$', r'$\varepsilon_2$']), 
                     kind='hist', corner=False, height=1.5, diag_kws=dict(color='black'), plot_kws=dict(cmap='mono'))

    eps_x_rec, eps_y_rec = stats.twiss2D.loc[meas_index, ['eps_x', 'eps_y']].values
    eps_1_rec, eps_2_rec = stats.twiss4D.loc[meas_index, ['eps_1', 'eps_2']].values
    emittances_rec = [eps_x_rec, eps_y_rec, eps_1_rec, eps_2_rec]
    for i in range(4):
        for j in range(4):
            if i != j:
                g.axes[i, j].scatter(emittances_rec[j], emittances_rec[i], c='red', s=10)
            else:
                g.diag_axes[j].axvline(emittances_rec[j], c='red')
    plt.suptitle('meas_index = {}'.format(meas_index))
#     plt.tight_layout()
    plt.savefig('_output/figures/eps_rand_{}.png'.format(meas_index), facecolor='white', **savefig_kws)
    plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
capsize = 1.5
plt_kws = dict(marker=None, lw=0, elinewidth=1.2, capsize=capsize)
g1 = ax.errorbar(xvals, emittances_mean_list[:, 0], yerr=emittances_std_list[:, 0], **plt_kws)
g2 = ax.errorbar(xvals, emittances_mean_list[:, 1], yerr=emittances_std_list[:, 1], **plt_kws)
g3 = ax.errorbar(xvals, emittances_mean_list[:, 2], yerr=emittances_std_list[:, 2], **plt_kws)
g4 = ax.errorbar(xvals, emittances_mean_list[:, 3], yerr=emittances_std_list[:, 3], **plt_kws)
lkws = dict(marker='.')
lines = [
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[0], **lkws),
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[1], **lkws),
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[2], **lkws),
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[3], **lkws),
]

# Plot the actual measured values.
ax.format(cycle='colorblind')
scatter_kws = dict(zorder=99, marker='.', capsize=capsize)
ax.errorbar(xvals, stats.twiss2D['eps_x'].values, yerr=0., **scatter_kws)
ax.errorbar(xvals, stats.twiss2D['eps_y'].values, yerr=0., **scatter_kws)
ax.errorbar(xvals, stats.twiss4D['eps_1'].values, yerr=0., **scatter_kws)
ax.errorbar(xvals, stats.twiss4D['eps_2'].values, yerr=0., **scatter_kws)

ax.legend(lines, labels=eps_labels, loc='upper left', ncols=1, handlelength=1.5)
ax.format(
    xlabel=xlabel, ylabel='[mm mrad]', 
    ylim=(0., ax.get_ylim()[1]), 
    xlim=(0, ax.get_xlim()[1]),
    **format_kws
)
plt.savefig('_output/figures/emittances_err.png', **savefig_kws)
plt.show()

In [ ]:
axes = myplt.rms_ellipses(Sigmas_mean_list, cmap=pplt.Colormap('flare'))
labels = ["x [mm]", "x' [mrad]", "y [mm]", "y' [mrad]"]
for i in range(3):
    axes[i, 0].set_ylabel(labels[i + 1])
    axes[-1, i].set_xlabel(labels[i])
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
plt_kws = dict(marker='.', capsize=1.5)
g1 = ax.errorbar(xvals, stats.twiss2D['beta_x'], yerr=twiss_stds[:, 2], **plt_kws)
g2 = ax.errorbar(xvals, stats.twiss2D['beta_y'], yerr=twiss_stds[:, 3], **plt_kws)
lkws = dict(marker='.')
lines = [
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[0], **lkws),
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[1], **lkws),
]
ax.legend(lines, labels=[r'$\beta_x$', r'$\beta_y$'], 
          loc='r', ncols=1, handlelength=2, framealpha=0.)
ax.format(xlabel=xlabel, ylabel='[m/rad]', **format_kws)
plt.savefig('_output/figures/beta.png', **savefig_kws)

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
plt_kws = dict(marker='.', capsize=1.5)
g1 = ax.errorbar(xvals, stats.twiss2D['alpha_x'], yerr=twiss_stds[:, 0], **plt_kws)
g2 = ax.errorbar(xvals, stats.twiss2D['alpha_y'], yerr=twiss_stds[:, 1], **plt_kws)
lkws = dict(marker='.')
lines = [
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[0], **lkws),
    Line2D([0], [0], color=myplt.DEFAULT_COLORCYCLE[1], **lkws),
]
ax.legend(lines, labels=[r'$\alpha_x$', r'$\alpha_y$'], 
          loc='r', ncols=1, handlelength=2, framealpha=0.)
ax.format(xlabel=xlabel, ylabel='[rad]', **format_kws)
plt.savefig('_output/figures/alpha.png', **savefig_kws)

In [ ]:
fig, ax = pplt.subplots(figsize=(3.5, 2.5))
eps_4D_apparent = stats.twiss2D['eps_x'] * stats.twiss2D['eps_y']
eps_4D = stats.twiss4D['eps_1'] * stats.twiss4D['eps_2']
g1 = ax.errorbar(xvals, eps_4D_apparent, yerr=eps_4D_apparent_stds, color='red', **plt_kws)
g2 = ax.errorbar(xvals, eps_4D, yerr=eps_4D_stds, color='blue', **plt_kws)
lines = [
    Line2D([0], [0], color='red', **lkws),
    Line2D([0], [0], color='blue', **lkws),
]
ax.legend(lines, labels=[''.join(eps_labels[:2]), ''.join(eps_labels[2:])], 
          loc='r', ncols=1, framealpha=0.)
ax.format(xlabel=xlabel, ylabel=r'[mm$^2$ mrad$^2$]', **format_kws)
plt.savefig('_output/figures/emittances4D.png', **savefig_kws)

In [ ]:
eps_x = stats.twiss2D['eps_x']
eps_y = stats.twiss2D['eps_y']
eps_1 = stats.twiss4D['eps_1']
eps_2 = stats.twiss4D['eps_2']
C = 1.0 - (eps_1 * eps_2) / (eps_x * eps_y)

fig, ax = pplt.subplots(figsize=(2.5, 2.5))
ax.plot(xvals, C, marker='.', color='k')
ax.grid(axis='y')
ax.format(xlabel='Turn number', 
          title=r'C = 1 - $\frac{\varepsilon_1\varepsilon_2}{\varepsilon_x\varepsilon_y}$',
          ylim=(0, 0.5), **format_kws)
plt.savefig('_output/figures/C.png', **savefig_kws)

In [ ]:
# for meas_index in meas_indices:

#     Sigma = Sigmas[meas_index]
#     axes = myplt.rms_ellipses(Sigma, color='black', pad=0.75, alpha=0.15, fill=True, lw=0, zorder=0)
#     plt.suptitle('Reconstructed phase space - turn {}'.format(n_turns[meas_index]), fontsize='medium')

#     _tmats_dict = dict()
#     _moments_dict = dict()
#     for ws_id in ws_ids:
#         _tmats_dict[ws_id] = tmats_dict[ws_id][meas_index:meas_index + 1]
#         _moments_dict[ws_id] = moments_dict[ws_id][meas_index:meas_index + 1]
#     reconstruction_lines(axes[2, 2], _tmats_dict, _moments_dict, plane='y-yp')
#     reconstruction_lines(axes[0, 0], _tmats_dict, _moments_dict, plane='x-xp',
#                          legend=True, legend_kws=dict(loc=(1.15, 0)))
#     plt.savefig('_output/figures/corner_ninjturns={}.png'.format(n_turns[meas_index]), **savefig_kws)
#     plt.show()

In [ ]:
# cmap = pplt.Colormap('flare')

# axes = myplt.rms_ellipses(Sigmas, 
#                           cmap=cmap
#                          )
# labels = ["x [mm]", "x' [mrad]", "y [mm]", "y' [mrad]"]
# for i in range(3):
#     axes[i, 0].set_ylabel(labels[i + 1])
#     axes[-1, i].set_xlabel(labels[i])
# plt.savefig('_output/figures/corner_all.png', **savefig_kws)

## Target image 

In [ ]:
from target_image import target_image_analysis as tis

In [ ]:
images = tis.read_files(
    utils.list_files(os.path.join(folder, 'target_images')), 
    make_square=False, # normal image follows target dimensions
#     thresh=2000, # image arrays must sum to at least `thresh` to be included (avoid blanks)
    thresh=0,
    n_avg=10 # number of images to average for each final image
)

In [ ]:
sigma = 4.0
plot_kws = dict(cmap='mono_r')
ellipse_kws = dict(lw=0.3, alpha=0.75)
n_turns = xvals

for i in range(len(Sigmas)):
    image = images[i]
    image.filter(sigma)

    fig, ax = pplt.subplots(figsize=(4.2, 2.0))
    
    # Plot the filtered image.
    ax.pcolormesh(image.xx, image.yy, image.Zf.T, **plot_kws)

    # Plot wire-scanner measurement. This assumes the wire-scanner reconstruction is
    # at the target. 
    mean_x, mean_y, sig_xx, sig_yy, sig_xy = image.estimate_moments(use_filtered=True)
    angle, c1, c2 = ba.rms_ellipse_dims(Sigmas[i])
    myplt.ellipse(ax, 2.0 * c1, 2.0 * c2, angle, center=(mean_x, mean_y), color='red8', **ellipse_kws)  
    
    # Plot the rms ellipse calculated from the image. This should be slightly wrong
    # due to the fiducial markers. We should do some interpolation at these points.
    angle, c1, c2 = ba._rms_ellipse_dims(sig_xx, sig_yy, sig_xy)
    myplt.ellipse(ax, 2.0 * c1, 2.0 * c2, angle, center=(mean_x, mean_y), color='blue8', **ellipse_kws)  

    # Formatting
    scale = 1.0
    xmax = scale * np.max(image.xx)
    ymax = scale * np.max(image.yy)
    ax.format(aspect=1.0, xlabel='x [mm]', ylabel='y [mm]',
              xlim=(-xmax, xmax), ylim=(-ymax, ymax),
              title='Turn {}'.format(n_turns[i])
             )
    plt.savefig('_output/figures/target_image_{}.png'.format(i), **savefig_kws)
    plt.show()

Set `vmax_manual` if you want to compare beam images for different setups. This will allow the colormap to represent the same thing for each image.

In [ ]:
vmax_manual = None 

In [ ]:
import time
start_time = time.time()

scale = 1.0
hist = False
sigma = 4.0
plot_kws = dict(cmap='mono_r', colorbar=True, colorbar_kw=dict(width=0.05, space=-2))
marginal_kws = dict(color='k')
ellipse_kws = dict(color='red', alpha=0.25, lw=0.25)

rms_ellipse = False
ws_ellipse = False

for same_ylim in [True, False]:
    hmax = vmax = None
    if same_ylim:
        hmax = vmax = 0.
        for image in images:
            Z = image.filter(sigma).T
            hmax = max(hmax, np.max(np.sum(Z, axis=0)))
            hmax = max(hmax, np.max(np.sum(Z, axis=1)))
            vmax = max(vmax, np.max(Z))
    if vmax_manual is not None:
        vmax = vmax_manual

    for i, image in enumerate(images):
        fig, ax = pplt.subplots(figsize=(4.25, 2.25), aspect=1)
        ax.pcolormesh(image.xx, image.yy, image.filter(sigma).T, vmax=vmax, **plot_kws)
        ax.format(xlabel='x [mm]', ylabel='y [mm]')
        ax.set_title('Turn = {}'.format(n_turns[i]))
        
        if ws_ellipse:
            angle, cx, cy = ba.rms_ellipse_dims(Sigmas[i])
            mean_x, mean_y, sig_xx, sig_yy, sig_xy = image.estimate_moments(use_filtered=True)
            myplt.ellipse(ax, 2.0 * cx, 2.0 * cy, angle, center=(mean_x, mean_y), **ellipse_kws) 
        
        if rms_ellipse:
            cx, cy, angle = target_image_analysis.rms_ellipse_dims(sig_xx, sig_yy, sig_xy)
            myplt.ellipse(ax, 2.0 * cx, 2.0 * cy, angle, center=(mean_x, mean_y), color='blue5', alpha=0.3, lw=0.25) 

        xmax = scale * np.max(image.xx)
        ymax = scale * np.max(image.yy)
        ax.format(xlim=(-xmax, xmax), ylim=(-ymax, ymax), aspect=1.0)

        figname = '_output/figures/targetimage_hist={}_sameylim={}_sigma{}_turn{}.png'.format(hist, same_ylim, sigma, n_turns[i])
        plt.savefig(figname, **savefig_kws)
        plt.close()

    print('Runtime = {} [seconds]'.format(time.time() - start_time))

In [ ]:
# print('vmax = {}'.format(vmax))

In [ ]:
# arrays = np.loadtxt('./_saved/2021-10-21/setting1/injturns300/bcm.dat')
# arrays = arrays[:, :100]

# fig, ax = pplt.subplots()
# # ax.pcolormesh(arrays[2:, :], colorbar=True, cmap='viridis')
# for array in arrays[2:]:
#     ax.plot(np.linspace(0, 1, len(array)), array, color='black')
# ax.format(xlabel='Fraction of ring', 
#           ylabel=r'Time $\rightarrow$')
# plt.savefig('/Users/46h/Research/long.png', **savefig_kws)
# plt.show()

In [ ]:
emittances_21mm = np.array([
    [5.16, 5.21, 5.42, 4.94], # 300 turns
    [6.79, 6.55, 7.44, 5.84], # 400 turns
    [7.95, 8.34, 9.52, 6.77], # 500 turns
])

emittances_31mm = np.array([
    [6.76, 6.29, 7.30, 5.70], # 300 turns
    [9.14, 7.72, 11.77, 4.73], # 400 turns
    [10.89, 8.52, 12.21, 7.09], # 500 turns
])

intensities = [0.3, 0.4, 0.5]

fig, ax = pplt.subplots()
plot_kws = dict(marker='.')
ax.plot(intensities, emittances_21mm, marker='.', lw=1)
ax.format(xlabel='Beam intensity')
plt.show()

fig, ax = pplt.subplots()
plot_kws = dict(marker='.')
ax.plot(intensities, emittances_31mm, marker='.', lw=1)
ax.format(xlabel='Beam intensity')
plt.show()

In [ ]:
for emittances, label in zip([emittances_21mm, emittances_31mm], ['21mm', '31mm']):
    fig, ax = pplt.subplots()
    x = intensities
    gap = abs(np.diff(x)[0])
    width = 0.1 * gap
    barwidth = 0.1
    rects1 = ax.bar(x - 1.5 * width, emittances[:, 0], barwidth, label=r'$\varepsilon_x$')
    rects2 = ax.bar(x - 0.5 * width, emittances[:, 1], barwidth, label=r'$\varepsilon_y$')
    rects3 = ax.bar(x + 0.5 * width, emittances[:, 2], barwidth, label=r'$\varepsilon_1$')
    rects4 = ax.bar(x + 1.5 * width, emittances[:, 3], barwidth, label=r'$\varepsilon_2$')
    ax.format(xticks=intensities, xtickminor=False, xlabel='Intensity / 1.5e14',
              ylabel='[mm mrad]', ylim=(0., 13.0)
             )
    ax.legend(ncols=2, loc=(0.0, 1.01), framealpha=0.)
    plt.savefig('_output/figures/intensity_{}.png'.format(label), **savefig_kws)